In [76]:
# Imports

import pandas as pd
import numpy as np
import json

import snowflake.connector
from sqlalchemy import create_engine

## RECS Data

### Reading RECS Data

In [77]:
# Read RECS data file

recs_data = pd.read_csv("recs2020_public_v7.csv")

In [78]:
recs_data.head()

,DOEID,REGIONC,DIVISION,STATE_FIPS,state_postal,state_name,BA_climate,IECC_climate_code,UATYP10,HDD65,...,EVCHRGHOME,EVCHRGAPT,EVCHRGWKS,EVCHRGBUS,EVCHRGMUNI,EVCHRGDLR,EVCHRGHWY,EVCHRGOTH,EVHOMEAMT,EVCHRGTYPE
0,100001,WEST,Mountain South,35,NM,New Mexico,Mixed-Dry,4B,U,3844,...,-2.0,-2,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0
1,100002,SOUTH,West South Central,5,AR,Arkansas,Mixed-Humid,4A,U,3766,...,-2.0,-2,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0
2,100003,WEST,Mountain South,35,NM,New Mexico,Mixed-Dry,4B,U,3819,...,-2.0,-2,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0
3,100004,SOUTH,South Atlantic,45,SC,South Carolina,Mixed-Humid,3A,U,2614,...,-2.0,-2,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0
4,100005,NORTHEAST,Middle Atlantic,34,NJ,New Jersey,Mixed-Humid,4A,U,4219,...,-2.0,-2,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0


### Validation Checks

In [79]:
# Validation Check 1: Checking for duplicated records and missing values

# Duplicate records check
duplicate_records = recs_data.duplicated().sum()
assert duplicate_records == 0, f"Dataset contains {duplicate_records} duplicate records"
recs_data.drop_duplicates(inplace=True)

# Missing values check
missing_values = recs_data.isnull().sum()
print("Missing values per column:\n", missing_values)
recs_data.dropna(inplace=True)

Missing values per column:
 DOEID            0
REGIONC          0
DIVISION         0
STATE_FIPS       0
state_postal     0
                ..
EVCHRGDLR       75
EVCHRGHWY       72
EVCHRGOTH       64
EVHOMEAMT       10
EVCHRGTYPE       9
Length: 799, dtype: int64


In [80]:
# Validation Check 2: Checking for outliers in Heating Degree Days

assert recs_data['HDD30YR_PUB'].between(0, 16071).all(), "HDD30YR_PUB values should be between 0 and 16071"
recs_data=recs_data[(recs_data['HDD30YR_PUB']>=0) & (recs_data['HDD30YR_PUB']<=16071)]

In [81]:
# Validation Check 3: Checking for geographical consistency

valid_states = {"AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY", 
                "LA", "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH", 
                "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"}
assert set(recs_data['state_postal'].unique()).issubset(valid_states), "Invalid states found in the dataset"
recs_data = recs_data[recs_data['state_postal'].isin(valid_states)]

In [84]:
# Validation Check 4: Checking data types

def check_dtypes(recs_data, variable_name, datatype):
    assert recs_data[variable_name].dtypes == datatype, f"{variable_name} should be an integer"
    recs_data[variable_name].astype(datatype).dtypes
    return recs_data

recs_data = check_dtypes(recs_data, 'TYPEHUQ', 'int64')
recs_data = check_dtypes(recs_data, 'WALLTYPE', 'int64')
recs_data = check_dtypes(recs_data, 'ACEQUIPM_PUB', 'int64')
recs_data = check_dtypes(recs_data, 'FUELHEAT', 'int64')
recs_data = check_dtypes(recs_data, 'HDD30YR_PUB', 'int64')

In [86]:
# Validation Check 5: Checking for incorrect categorical values

def check_values(recs_data, variable_name, valid_values):
    assert set(recs_data[variable_name].unique()).issubset(valid_values), f"Invalid {variable_name} values found"
    recs_data = recs_data[(recs_data[variable_name].isin(valid_values))]
    return recs_data

recs_data = check_values(recs_data, 'TYPEHUQ', [1, 2, 3, 4, 5])
recs_data = check_values(recs_data, 'WALLTYPE', [1, 2, 3, 4, 5, 6, 7, 99])
recs_data = check_values(recs_data, 'ACEQUIPM_PUB', [1, -2, 3, 4, 5, 6,])
recs_data = check_values(recs_data, 'FUELHEAT', [1, 2, 3, 5, 7, 99, -2])

### Ingesting Data into Database by Creating Connection to Snowflake

In [87]:
# Automate SQL code generation for creating RECS_DATA table

dtypes_df = recs_data.dtypes.to_frame('dtypes').reset_index()
dtypes_df['dtypes'] = dtypes_df['dtypes'].replace('int64', 'INT')
dtypes_df['dtypes'] = dtypes_df['dtypes'].replace('float64', 'FLOAT')
dtypes_df['dtypes'] = dtypes_df['dtypes'].replace('object', 'VARCHAR(255)')
dtypes_df['dtypes'] = dtypes_df['dtypes'].replace('O', 'VARCHAR(255)')

dtypes_df['sql_cde'] = dtypes_df['index'] + ' ' + dtypes_df['dtypes'] + ', '

col_names = ''

for i in dtypes_df['sql_cde']:
    col_names += i
sql_create = f'CREATE OR REPLACE TABLE RECS_DB.RECS_DATA_SCHEMA.RECS_DATA ({col_names});'

In [ ]:
# Ingest data into RECS_DATA table

with open('config.json', 'r') as config_file:
    config_data = json.load(config_file)

user="SindhuSwaroop"
role="accountadmin"
warehouse="compute_wh"
database="RECS_DB"
schema="recs_data_schema"
account = "rhjhrje-st87781"
password = config_data.get("password")

conn=snowflake.connector.connect(user=user, 
                                 role=role, 
                                 warehouse=warehouse, 
                                 database=database, 
                                 schema=schema, 
                                 account=account, 
                                 password=password)

engine = create_engine(
    f'snowflake://{user}:{password}@{account}/{database}/{schema}?warehouse={warehouse}'
)

recs_data.to_sql(name='recs_data', con=engine, if_exists='replace', index=False, index_label=None)

## Codes Data

### Reading Codebook

In [88]:
# Read Codebook

codes_data = pd.read_excel("RECS 2020 Codebook for Public File - v7.xlsx", header=1, sheet_name='codebook')

### Creating Connection to Codes Schema in Snowflake

In [89]:
# Create connection to codes_schema on Snowflake

schema="codes_schema"

conn=snowflake.connector.connect(user=user, 
                                 role=role, 
                                 warehouse=warehouse, 
                                 database=database, 
                                 schema=schema, 
                                 account=account, 
                                 password=password)

engine = create_engine(
    f'snowflake://{user}:{password}@{account}/{database}/{schema}?warehouse={warehouse}'
)

### Creating Type Tables for Required Variables and Ingesting Data into Database

In [64]:
# Create type table for HDD30YR_PUB

code_HDD30YR_PUB = codes_data[codes_data['Variable']=="HDD30YR_PUB"]

# Expand the rows
expanded_rows = []

for _, row in code_HDD30YR_PUB.iterrows():
    variable = row['Variable']
    description = row['Description and Labels']
    response_code_range = row['Response Codes']
    
    start, end = map(int, response_code_range.split('-'))
    
    for code in range(start, end + 1):
        expanded_rows.append([variable, description, code])

expanded_df = pd.DataFrame(expanded_rows, columns=['variable_name', 'description', 'code'])
expanded_df.to_sql(name='TYPE_HDD30YR_PUB', con=engine, if_exists='append', index=False, index_label=None)

/var/folders/g6/zzp2wzs96_3b8mvqcqp1hbcw0000gn/T/ipykernel_19371/3619588838.py:19: UserWarning: The provided table name 'TYPE_HDD30YR_PUB' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  expanded_df.to_sql(name='TYPE_HDD30YR_PUB', con=engine, if_exists='append', index=False, index_label=None)


16072

In [65]:
# Create type table for TYPEHUQ

code_TYPEHUQ = codes_data[codes_data['Variable']=="TYPEHUQ"]

# Expand the rows
expanded_rows = []

for _, row in code_TYPEHUQ.iterrows():
    variable = row['Variable']
    description = row['Description and Labels']
    response_codes = row['Response Codes'].split('\n')
    
    for code_desc in response_codes:
        code, desc = code_desc.split(' ', 1)
        expanded_rows.append([variable, description, code, desc])

expanded_df = pd.DataFrame(expanded_rows, columns=['variable_name', 'description', 'code', 'code_desc'])
expanded_df.to_sql(name='TYPE_TYPEHUQ', con=engine, if_exists='append', index=False, index_label=None)

/var/folders/g6/zzp2wzs96_3b8mvqcqp1hbcw0000gn/T/ipykernel_19371/4149091994.py:18: UserWarning: The provided table name 'TYPE_TYPEHUQ' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  expanded_df.to_sql(name='TYPE_TYPEHUQ', con=engine, if_exists='append', index=False, index_label=None)


5

In [66]:
# Create type table for WALLTYPE

code_WALLTYPE = codes_data[codes_data['Variable']=="WALLTYPE"]

# Expand the rows
expanded_rows = []

for _, row in code_WALLTYPE.iterrows():
    variable = row['Variable']
    description = row['Description and Labels']
    response_codes = row['Response Codes'].split('\n')
    
    for code_desc in response_codes:
        code, desc = code_desc.split(' ', 1)
        expanded_rows.append([variable, description, code, desc])

expanded_df = pd.DataFrame(expanded_rows, columns=['variable_name', 'description', 'code', 'code_desc'])
expanded_df.to_sql(name='TYPE_WALLTYPE', con=engine, if_exists='append', index=False, index_label=None)

/var/folders/g6/zzp2wzs96_3b8mvqcqp1hbcw0000gn/T/ipykernel_19371/1910766460.py:18: UserWarning: The provided table name 'TYPE_WALLTYPE' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  expanded_df.to_sql(name='TYPE_WALLTYPE', con=engine, if_exists='append', index=False, index_label=None)


8

In [67]:
# Create type table for ACEQUIPM_PUB

code_ACEQUIPM_PUB = codes_data[codes_data['Variable']=="ACEQUIPM_PUB"]

# Expand the rows
expanded_rows = []

for _, row in code_ACEQUIPM_PUB.iterrows():
    variable = row['Variable']
    description = row['Description and Labels']
    response_codes = row['Response Codes'].split('\n')
    
    for code_desc in response_codes:
        code, desc = code_desc.split(' ', 1)
        expanded_rows.append([variable, description, code, desc])

expanded_df = pd.DataFrame(expanded_rows, columns=['variable_name', 'description', 'code', 'code_desc'])
expanded_df.to_sql(name='TYPE_ACEQUIPM_PUB', con=engine, if_exists='append', index=False, index_label=None)

/var/folders/g6/zzp2wzs96_3b8mvqcqp1hbcw0000gn/T/ipykernel_19371/30216171.py:18: UserWarning: The provided table name 'TYPE_ACEQUIPM_PUB' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  expanded_df.to_sql(name='TYPE_ACEQUIPM_PUB', con=engine, if_exists='append', index=False, index_label=None)


6

In [68]:
# Create type table for FUELHEAT

code_FUELHEAT = codes_data[codes_data['Variable']=="FUELHEAT"]

# Expand the rows
expanded_rows = []

for _, row in code_FUELHEAT.iterrows():
    variable = row['Variable']
    description = row['Description and Labels']
    response_codes = row['Response Codes'].split('\n')
    
    for code_desc in response_codes:
        code, desc = code_desc.split(' ', 1)
        expanded_rows.append([variable, description, code, desc])

expanded_df = pd.DataFrame(expanded_rows, columns=['variable_name', 'description', 'code', 'code_desc'])
expanded_df.to_sql(name='TYPE_FUELHEAT', con=engine, if_exists='append', index=False, index_label=None)

/var/folders/g6/zzp2wzs96_3b8mvqcqp1hbcw0000gn/T/ipykernel_19371/660827295.py:18: UserWarning: The provided table name 'TYPE_FUELHEAT' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  expanded_df.to_sql(name='TYPE_FUELHEAT', con=engine, if_exists='append', index=False, index_label=None)


7

### Reading State Dictionary

In [70]:
# Read State Dictionary

state_data = pd.read_excel("RECS 2020 Codebook for Public File - v7.xlsx", sheet_name='state_dictionary')

### Creating States Table and Ingesting Data into Database

In [71]:
# Create states table

state_data.to_sql(name='STATES', con=engine, if_exists='append', index=False, index_label=None)

/var/folders/g6/zzp2wzs96_3b8mvqcqp1hbcw0000gn/T/ipykernel_19371/404408270.py:3: UserWarning: The provided table name 'STATES' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  state_data.to_sql(name='STATES', con=engine, if_exists='append', index=False, index_label=None)


51